In [1]:
import pandas as pd
import pymongo
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# conn = psycopg2.connect(database="postgres", user='postgres', password='Shjais2014', host='127.0.0.1', port='5432')
# conn.autocommit = True

In [3]:
csv_file = "./Data/breweries.csv"
breweries_df = pd.read_csv(csv_file)
breweries_df.head()

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [4]:
breweries_df['state'] = breweries_df['state'].str.replace(' ', '')

In [5]:
breweries_df.head()

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [6]:
breweries_df['state'].value_counts()

CO    47
CA    39
MI    32
OR    29
TX    28
PA    25
WA    23
MA    23
IN    22
WI    20
NC    19
IL    18
NY    16
VA    16
FL    15
OH    15
MN    12
AZ    11
VT    10
MO     9
MT     9
ME     9
CT     8
MD     7
GA     7
AK     7
OK     6
NE     5
IA     5
ID     5
LA     5
RI     5
HI     4
NM     4
UT     4
KY     4
SC     4
WY     4
TN     3
AL     3
KS     3
NH     3
NJ     3
MS     2
AR     2
NV     2
DE     2
SD     1
WV     1
DC     1
ND     1
Name: state, dtype: int64

In [7]:
breweries_df.replace(to_replace = "DC", value = "District of Columbia", inplace = True)
breweries_df['state'].value_counts()

CO                      47
CA                      39
MI                      32
OR                      29
TX                      28
PA                      25
WA                      23
MA                      23
IN                      22
WI                      20
NC                      19
IL                      18
VA                      16
NY                      16
OH                      15
FL                      15
MN                      12
AZ                      11
VT                      10
MT                       9
ME                       9
MO                       9
CT                       8
MD                       7
AK                       7
GA                       7
OK                       6
ID                       5
LA                       5
NE                       5
IA                       5
RI                       5
HI                       4
NM                       4
UT                       4
SC                       4
WY                       4
K

In [8]:
breweries_df = breweries_df.drop(columns=['Unnamed: 0', 'city']) 

In [9]:
breweries_df = breweries_df.groupby(['state']).count()

In [10]:
breweries_df.head(10)

,name
state,
AK,7
AL,3
AR,2
AZ,11
CA,39
CO,47
CT,8
DE,2
District of Columbia,1


In [11]:
csv_file = "./Data/Costs of Excessive Alcohol Consumption by State.csv"
costs_df = pd.read_csv(csv_file)
costs_df.head()

,State,Location,Total Cost ($),Cost per drink ($),Cost per capita ($)
0,AL,Alabama,"3,724,300,000",2.27,779
1,AK,Alaska,"827,200,000",2.25,"1,165"
2,AZ,Arizona,"5,946,400,000",2.27,930
3,AR,Arkansas,"2,073,300,000",2.27,711
4,CA,California,"35,010,600,000",2.44,940


In [12]:
costs_df = costs_df.drop(columns=['Location']) 
costs_df.head()

,State,Total Cost ($),Cost per drink ($),Cost per capita ($)
0,AL,"3,724,300,000",2.27,779
1,AK,"827,200,000",2.25,"1,165"
2,AZ,"5,946,400,000",2.27,930
3,AR,"2,073,300,000",2.27,711
4,CA,"35,010,600,000",2.44,940


In [13]:
costs_df.rename(columns = {'State':'state', 'Total Cost ($)':'total_cost', 'Cost per drink ($)':'cost_per_drink',
                          'Cost per capita ($)':'cost_per_capita'}, inplace = True) 
costs_df.head()

,state,total_cost,cost_per_drink,cost_per_capita
0,AL,"3,724,300,000",2.27,779
1,AK,"827,200,000",2.25,"1,165"
2,AZ,"5,946,400,000",2.27,930
3,AR,"2,073,300,000",2.27,711
4,CA,"35,010,600,000",2.44,940


In [14]:
costs_df = costs_df.set_index("state")
costs_df.head()

,total_cost,cost_per_drink,cost_per_capita
state,,,
AL,"3,724,300,000",2.27,779
AK,"827,200,000",2.25,"1,165"
AZ,"5,946,400,000",2.27,930
AR,"2,073,300,000",2.27,711
CA,"35,010,600,000",2.44,940


In [15]:
costs_df['total_cost'] = costs_df['total_cost'].str.replace(',', '')
costs_df['cost_per_capita'] = costs_df['cost_per_capita'].str.replace(',', '')
costs_df.head()

,total_cost,cost_per_drink,cost_per_capita
state,,,
AL,3724300000,2.27,779
AK,827200000,2.25,1165
AZ,5946400000,2.27,930
AR,2073300000,2.27,711
CA,35010600000,2.44,940


In [16]:
costs_df.rename(columns={"total_cost": " Total Cost", "cost_per_drink": "Cost per Drink", "cost_per_capita": "Cost per Capita"})

,Total Cost,Cost per Drink,Cost per Capita
state,,,
AL,3724300000,2.27,779
AK,827200000,2.25,1165
AZ,5946400000,2.27,930
AR,2073300000,2.27,711
CA,35010600000,2.44,940
CO,5056500000,2.14,1005
CT,3029000000,2.04,847
DE,803800000,1.64,895
District of Columbia,918400000,2.14,1526


In [17]:
csv_file = "./Data/Intensity of Binge Drinking.csv"
intensity_df = pd.read_csv(csv_file)
intensity_df.head()

,State,Percentage
0,AL,7.3
1,AK,7.8
2,AZ,7.1
3,AR,8.3
4,CA,6.5


In [18]:
# rename columns in intensity_df, the website, https://www.cdc.gov/alcohol/data-stats.htm, says this is not a percentage but
# the average largest number of drinks consumed by binge drinkers on any occasion
intensity_df.rename(columns = {'State':'state', 'Percentage':'avg_num_of_drinks'}, inplace = True) 
intensity_df.head()

,state,avg_num_of_drinks
0,AL,7.3
1,AK,7.8
2,AZ,7.1
3,AR,8.3
4,CA,6.5


In [19]:
intensity_df.rename(columns = {'avg_num_of_drinks' : 'Avg number of Drinks'}, inplace = True)
(intensity_df)

,state,Avg number of Drinks
0,AL,7.3
1,AK,7.8
2,AZ,7.1
3,AR,8.3
4,CA,6.5
5,CO,6.6
6,CT,7.0
7,DE,7.0
8,District of Columbia,6.2
9,FL,6.8


In [20]:
intensity_df = intensity_df.set_index("state")
intensity_df.head()

,Avg number of Drinks
state,
AL,7.3
AK,7.8
AZ,7.1
AR,8.3
CA,6.5


In [21]:
csv_file = "./Data/Prevalence of Binge Drinking.csv"
prevalence_df = pd.read_csv(csv_file)
prevalence_df.head()

,State,Percentage
0,AL,12.2
1,AK,20.0
2,AZ,15.0
3,AR,15.2
4,CA,16.7


In [22]:
prevalence_df.rename(columns = {'State':'state', 'Percentage':'percentage'}, inplace = True) 
prevalence_df.head()

,state,percentage
0,AL,12.2
1,AK,20.0
2,AZ,15.0
3,AR,15.2
4,CA,16.7


In [23]:
selected_column = prevalence_df[["state"]]

In [24]:
prevalence_df = prevalence_df.set_index("state")
prevalence_df.head()

,percentage
state,
AL,12.2
AK,20.0
AZ,15.0
AR,15.2
CA,16.7


In [25]:
csv_file = "./Data/Health.csv"
health_df = pd.read_csv(csv_file)
health_df.head()

,Location,All Adults,Male,Female,Footnotes
0,United States,18.80%,18.20%,19.40%,1.0
1,Alabama,22.70%,21.60%,23.80%,NaN
2,Alaska,15.50%,15.30%,15.80%,NaN
3,Arizona,19.00%,17.40%,20.50%,NaN
4,Arkansas,25.20%,23.90%,26.30%,NaN


In [26]:
health_df = health_df.drop([0]).reset_index(drop = True)
health_df.head()

,Location,All Adults,Male,Female,Footnotes
0,Alabama,22.70%,21.60%,23.80%,NaN
1,Alaska,15.50%,15.30%,15.80%,NaN
2,Arizona,19.00%,17.40%,20.50%,NaN
3,Arkansas,25.20%,23.90%,26.30%,NaN
4,California,18.20%,18.50%,18.00%,NaN


In [27]:
health_df = health_df.join(selected_column)

In [28]:
health_df.head()

,Location,All Adults,Male,Female,Footnotes,state
0,Alabama,22.70%,21.60%,23.80%,NaN,AL
1,Alaska,15.50%,15.30%,15.80%,NaN,AK
2,Arizona,19.00%,17.40%,20.50%,NaN,AZ
3,Arkansas,25.20%,23.90%,26.30%,NaN,AR
4,California,18.20%,18.50%,18.00%,NaN,CA


In [29]:
health_df = health_df.drop(columns=['Location', 'Male', 'Female', 'Footnotes'])
health_df.head() 

,All Adults,state
0,22.70%,AL
1,15.50%,AK
2,19.00%,AZ
3,25.20%,AR
4,18.20%,CA


In [30]:
health_df.rename(columns = {'All Adults':'health_percent'}, inplace = True) 
health_df.head()

,health_percent,state
0,22.70%,AL
1,15.50%,AK
2,19.00%,AZ
3,25.20%,AR
4,18.20%,CA


In [31]:
health_df['health_percent'] = health_df['health_percent'].str.replace('%', '')
health_df.head()


,health_percent,state
0,22.70,AL
1,15.50,AK
2,19.00,AZ
3,25.20,AR
4,18.20,CA


In [32]:
health_df['health_percent'] = health_df['health_percent'].astype(float)
health_df.head()

,health_percent,state
0,22.7,AL
1,15.5,AK
2,19.0,AZ
3,25.2,AR
4,18.2,CA


In [33]:
health_df.rename(columns={"health_percent": " Health Percent"})

,Health Percent,state
0,22.7,AL
1,15.5,AK
2,19.0,AZ
3,25.2,AR
4,18.2,CA
5,14.0,CO
6,14.9,CT
7,18.7,DE
8,13.6,District of Columbia
9,19.7,FL


In [34]:
health_df = health_df.set_index("state")
health_df.head()



,health_percent
state,
AL,22.7
AK,15.5
AZ,19.0
AR,25.2
CA,18.2


In [35]:
health_df.rename(columns={"health_percent": " Health Percent"})

,Health Percent
state,
AL,22.7
AK,15.5
AZ,19.0
AR,25.2
CA,18.2
CO,14.0
CT,14.9
DE,18.7
District of Columbia,13.6


In [36]:
conn = psycopg2.connect(database="Alcohol", user='postgres', password='Shjais2014', host='127.0.0.1', port='5432')
conn.autocommit = True
cursor = conn.cursor()

OperationalError: FATAL:  password authentication failed for user "postgres"


In [37]:
schema_sql = open('./sql/schema.sql','r')
cursor.execute(schema_sql.read())
print("Database Schema Loaded Successfully (Tables Created)........")

NameError: name 'cursor' is not defined

In [ ]:
schema_sql.close()

In [ ]:
#database connection
connection_string ="postgres:Shjais2014@localhost:5432/Alcohol"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
#confirm tables
engine.table_names()

In [ ]:
### Load DataFrames into database
breweries_df.to_sql(name='breweries', con=engine, if_exists='append', index = True)

In [ ]:
costs_df.to_sql(name='costs', con=engine, if_exists='append', index = True)

In [ ]:
intensity_df.to_sql(name='intensity', con=engine, if_exists='append', index=True)

In [ ]:
prevalence_df.to_sql(name='prevalence', con=engine, if_exists='append', index = True)

In [ ]:
health_df.to_sql(name='health', con=engine, if_exists='append', index=True)

In [ ]:
join_sql = open('sql/schema_2.sql','r')
alcohol_df = pd.read_sql_query(join_sql.read(), con=engine)
alcohol_df

In [ ]:
alcohol_df.to_csv('./Data/merged_data.csv')

In [ ]:
#Close File after use
join_sql.close()